In [101]:
from datasets import load_dataset
from pprint import pprint
from collections import Counter
import pandas as pd
from datasets import Dataset
from unicodedata import normalize, is_normalized
from spacy_alignments.tokenizations import get_alignments
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer,BatchEncoding, pipeline
import torch
from seqeval.metrics.sequence_labeling import get_entities

In [94]:
dataset = load_dataset('llm-book/ner-wikipedia-dataset', trust_remote_code=True)

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 4274
    })
    validation: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 534
    })
    test: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 535
    })
})


In [4]:
pprint(list(dataset['train'])[:2])

[{'curid': '3638038',
  'entities': [{'name': 'さくら学院', 'span': [0, 5], 'type': 'その他の組織名'},
               {'name': 'Ciao Smiles', 'span': [6, 17], 'type': 'その他の組織名'}],
  'text': 'さくら学院、Ciao Smilesのメンバー。'},
 {'curid': '1729527',
  'entities': [{'name': 'レクレアティーボ・ウェルバ', 'span': [17, 30], 'type': 'その他の組織名'},
               {'name': 'プリメーラ・ディビシオン', 'span': [32, 44], 'type': 'その他の組織名'}],
  'text': '2008年10月5日、アウェーでのレクレアティーボ・ウェルバ戦でプリメーラ・ディビシオンでの初得点を決めた。'}]


In [5]:
for i in dataset['train']:
    pprint(i)
    break

{'curid': '3638038',
 'entities': [{'name': 'さくら学院', 'span': [0, 5], 'type': 'その他の組織名'},
              {'name': 'Ciao Smiles', 'span': [6, 17], 'type': 'その他の組織名'}],
 'text': 'さくら学院、Ciao Smilesのメンバー。'}


・今回はテキストに含まれる固有表現のスパンとそのタイプを指定する

In [6]:
# データセットの分析

def count_label_occurrences(dataset: Dataset) -> dict[str, int]:

    # 固有表現タイプを抽出したlistを作成する
    entities = [
        e['type'] for data in dataset for e in data['entities']
    ]

    # ラベルの出現回数が多い順に並び変える
    # Counterにはmost_common()メソッドがあり、(要素, 出現回数)という形のタプルを出現回数順に並べたリストを返す。
    label_counts = dict(Counter(entities).most_common())
    return label_counts


label_counts_dict = {}
for split in dataset:
    label_counts_dict[split] = count_label_occurrences(dataset[split])
df = pd.DataFrame(label_counts_dict)
df.loc['合計'] = df.sum()
df

,train,validation,test
人名,2394,299,287
法人名,2006,231,248
地名,1769,184,204
政治的組織名,953,121,106
製品名,934,123,158
施設名,868,103,137
その他の組織名,852,99,100
イベント名,831,85,93
合計,10607,1245,1333


In [7]:
def has_overlap(spans):
    sorted_spans = sorted(spans, key=lambda x: x[0])
    for i in range(1, len(sorted_spans)):
        if sorted_spans[i-1][1] > sorted_spans[i][0]:
            return 1
    return 0


overlap_count = 0
for split in dataset:
    for data in dataset[split]:
        if data['entities']:
            spans = [e['span'] for e in data['entities']]
            overlap_count += has_overlap(spans)

    print(f"{split}におけるスパンが重複する事例数：{overlap_count}")

trainにおけるスパンが重複する事例数：0
validationにおけるスパンが重複する事例数：0
testにおけるスパンが重複する事例数：0


In [8]:
spans

[[0, 9], [10, 21], [25, 37]]

In [9]:
data['entities']

[{'name': 'ダーヴラ・カーワン', 'span': [0, 9], 'type': '人名'},
 {'name': 'マーシー・ハーティガン', 'span': [10, 21], 'type': '人名'},
 {'name': 'ラッセル・T・デイヴィス', 'span': [25, 37], 'type': '人名'}]

In [10]:
dataset['test'][-1:]

{'curid': ['4113413'],
 'text': ['ダーヴラ・カーワンはマーシー・ハーティガンを演じ、ラッセル・T・デイヴィスは本作のポッドキャストコメンタリーで彼女について「これまでにないほどダークな悪役」と表現した。'],
 'entities': [[{'name': 'ダーヴラ・カーワン', 'span': [0, 9], 'type': '人名'},
   {'name': 'マーシー・ハーティガン', 'span': [10, 21], 'type': '人名'},
   {'name': 'ラッセル・T・デイヴィス', 'span': [25, 37], 'type': '人名'}]]}

## 前処理

## テキスト正規化

In [11]:
text = "ABCＡＢＣabcABCアイウｱｲｳ①②③123"

nomalized_text = normalize('NFKC', text)
print('正規化前', text)
print('正規化後', nomalized_text)

正規化前 ABCＡＢＣabcABCアイウｱｲｳ①②③123
正規化後 ABCABCabcABCアイウアイウ123123


In [12]:
count = 0
for split in dataset:
    for data in dataset[split]:
        if not is_normalized('NFKC',data['text']): # 正規化されていないとFalseをかえす？
            count += 1
print(f'正規化されていない事例数: {count}')

正規化されていない事例数: 0


In [13]:
text = "ABCＡＢＣabcABCアイウｱｲｳ①②③123"
is_normalized('NFKC', text)

False

In [14]:
'/'.join(dataset['train'][0]['text'])

'さ/く/ら/学/院/、/C/i/a/o/ /S/m/i/l/e/s/の/メ/ン/バ/ー/。'

## 文字列とトークン列のアライメント

In [15]:
model_name = "tohoku-nlp/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)


text ='さくら学院'

# 文字列のLISTに変換
characters = list(text)

# 特殊トークンも含めたリストにする
tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))

char_to_token_indices, token_to_char_indices = get_alignments(characters, tokens)
print(characters, tokens)
print('文字に対するトークンの位置',char_to_token_indices)
print('トークンに対する文字の位置',token_to_char_indices)

['さ', 'く', 'ら', '学', '院'] ['[CLS]', 'さくら', '学院', '[SEP]']
文字に対するトークンの位置 [[1], [1], [1], [2], [2]]
トークンに対する文字の位置 [[], [0, 1, 2], [3, 4], []]


In [16]:
print(characters, tokens)

['さ', 'く', 'ら', '学', '院'] ['[CLS]', 'さくら', '学院', '[SEP]']


In [17]:
text = '大谷翔平は岩手県水沢市出身'
entities = [
    {'name':'大谷正平', 'span':[0, 4], 'type':'人名'},
    {'name':'岩手県水沢市', 'span':[5, 11], 'type':'地方'}
]

In [18]:



def output_tokens_and_labels(text, entities, tokenizer):
    characters = list(text)
    tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))
    char_to_token_indices, _ = get_alignments(characters, tokens)
    
    # 0で初期化したラベルリスト
    labels = ['0'] * len(tokens)
    for entity in entities:
        entity_span, entity_type = entity['span'], entity['type']
        start = char_to_token_indices[entity_span[0]][0]
        end = char_to_token_indices[entity_span[1] -1][0]
        labels[start] = f"B-{entity_type}"
        for idx in range(start + 1, end + 1):
            labels[idx] = f"I-{entity_type}"
    
        labels[0] = '-'
        labels[-1] = '-'
    return tokens, labels


tokens, labels = output_tokens_and_labels(text, entities, tokenizer)

df = pd.DataFrame({'トークン列':tokens, 'ラベル列':labels})
df.index.name = "位置"
df.T

位置,0,1,2,3,4,5,6,7,8,9,10
トークン列,[CLS],大谷,翔,##平,は,岩手,県,水沢,市,出身,[SEP]
ラベル列,-,B-人名,I-人名,I-人名,0,B-地方,I-地方,I-地方,I-地方,0,-


In [42]:
from typing import Any
from seqeval.metrics import classification_report

def create_character_labels(
    text: str, entities: list[dict[str, list[int] | str]]
) -> list[str]:
    """文字ベースでラベルのlistを作成"""
    # "O"のラベルで初期化したラベルのlistを作成する
    labels = ["O"] * len(text)
    for entity in entities: # 各固有表現を処理する
        entity_span, entity_type = entity["span"], entity["type"]
        # 固有表現の開始文字の位置に"B-"のラベルを設定する
        labels[entity_span[0]] = f"B-{entity_type}"
        # 固有表現の開始文字以外の位置に"I-"のラベルを設定する
        for i in range(entity_span[0] + 1, entity_span[1]):
            labels[i] = f"I-{entity_type}"
    return labels

def convert_results_to_labels(
    results: list[dict[str, Any]]
) -> tuple[list[list[str]], list[list[str]]]:
    """正解データと予測データのラベルのlistを作成"""
    true_labels, pred_labels = [], []
    for result in results: # 各事例を処理する
        # 文字ベースでラベルのリストを作成してlistに加える
        true_labels.append(
            create_character_labels(result["text"], result["entities"])
        )
        pred_labels.append(
            create_character_labels(result["text"], result["pred_entities"])
        )
    return true_labels, pred_labels

In [43]:
## 評価指標のseqevalの挙動

results = [
    {
        "text": "大谷翔平は岩手県水沢市出身",
        "entities": [
            {"name": "大谷翔平", "span": [0, 4], "type": "人名"},
            {"name": "岩手県水沢市", "span": [5, 11], "type": "地名"},
        ],
        "pred_entities": [
            {"name": "大谷翔平", "span": [0, 4], "type": "人名"},
            {"name": "岩手県", "span": [5, 8], "type": "地名"},
            {"name": "水沢市", "span": [8, 11], "type": "施設名"},
        ],
    }
]

true_labels, pred_labels = convert_results_to_labels(results)
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

          人名       1.00      1.00      1.00         1
          地名       0.00      0.00      0.00         1
         施設名       0.00      0.00      0.00         0

   micro avg       0.33      0.50      0.40         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2



In [44]:
from seqeval.metrics import f1_score, precision_score, recall_score

def compute_scores(true_labels: list[list[str]], pred_labels: list[list[str]], average:str) -> dict[str, float]:
    scores = {
        'precision': precision_score(true_labels, pred_labels, average=average),
        'recall': recall_score(true_labels, pred_labels, average=average),
        'F1-score': f1_score(true_labels, pred_labels, average=average),
    }

    return scores


print(compute_scores(true_labels, pred_labels, 'micro'))

{'precision': 0.3333333333333333, 'recall': 0.5, 'F1-score': 0.4}


## 固有表現認識モデルの実装

In [45]:
# BERTのファインチューニング

# label1id
def create_label2id(
    entities_list: list[list[dict[str, str | str]]]
) -> dict[str, int]:
    label2id = {"0": 0}

    # setなので重複はなし
    entity_type = set([e['type'] for entities in entities_list for e in entities])

    entity_types = sorted(entity_type)

    # 1entityにつき2種類登録
    for i, entity_type in enumerate(entity_types):
        label2id[f"B-{entity_type}"] = i*2 + 1
        label2id[f"I-{entity_type}"] = i*2 + 2
    return label2id


label2id = create_label2id(dataset['train']["entities"])
id2label = {id:v for v, id in label2id.items()}

In [100]:
# データの前処理


def preprocess_data(data, tokenizer, label2id) -> BatchEncoding:
    # トークナイゼーション
    inputs = tokenizer(data['text'], return_tensors='pt', return_special_tokens_mask=True)
    inputs = { k:v.squeeze(0) for k, v in inputs.items()}

    characters = list(data['text'])
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'])
    # print(characters)
    # print(tokens)
    char_to_token_indeces, _ = get_alignments(characters, tokens)

    labels = torch.zeros_like(inputs['input_ids'])
    for entity in data['entities']:
        # print(char_to_token_indeces)
        # print(entity['span'][0])
        # print(entity['span'][1] - 1)
        start_token_indeces = char_to_token_indeces[entity['span'][0]]
        end_token_indeces = char_to_token_indeces[entity['span'][1] - 1]

        # 文字に対応するトークンが存在しなければスキップ -> 固有表現ではない単語 ex(は)
        if(
            len(start_token_indeces) == 0
            or len(end_token_indeces) == 0
        ):
            continue

        start, end = start_token_indeces[0], end_token_indeces[0]
        # print(start, end)
        entity_type = entity['type']

        labels[start] = label2id[f"B-{entity_type}"]
        if start != end:
            labels[start + 1 : end + 1] = label2id[f"I-{entity_type}"]


    labels[torch.where(inputs["special_tokens_mask"])] = -100
    inputs['labels'] = labels
    return inputs
        
        

In [64]:
test = dataset['train'][0]
preprocess_data(test, tokenizer, label2id)

{'input_ids': tensor([    2, 16972, 14284,   384,    50, 13634,  7075, 20218, 18124,  7045,
           464, 12913,   385,     3]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'special_tokens_mask': tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor([-100,    1,    2,    0,    1,    2,    2,    2,    2,    2,    0,    0,
            0, -100])}

In [65]:
pprint(dataset['train'][0])

{'curid': '3638038',
 'entities': [{'name': 'さくら学院', 'span': [0, 5], 'type': 'その他の組織名'},
              {'name': 'Ciao Smiles', 'span': [6, 17], 'type': 'その他の組織名'}],
 'text': 'さくら学院、Ciao Smilesのメンバー。'}


In [121]:
# 訓練セットに前処理
train_dataset = dataset['train'].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer":tokenizer,
        "label2id":label2id
    }, remove_columns=dataset['train'].column_names
)

validation_dataset = dataset['validation'].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer":tokenizer,
        "label2id":label2id
    }, remove_columns=dataset['validation'].column_names
)

Map:   0%|          | 0/4274 [00:00<?, ? examples/s]

Map:   0%|          | 0/534 [00:00<?, ? examples/s]

In [122]:
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(model_name, label2id=label2id, id2label=id2label)
data_collator = DataCollatorForTokenClassification(tokenizer)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [123]:
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import set_seed

set_seed(42)

training_args = TrainingArguments(
    output_dir='output_bert_ner',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-4,
    lr_scheduler_type='linear',
    warmup_ratio=0.1,
    num_train_epochs=5,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='epoch',
    fp16=True
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    args=training_args
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.655500,0.101072
2,0.072300,0.083893
3,0.029900,0.086092
4,0.012300,0.094577
5,0.005900,0.098206


TrainOutput(global_step=670, training_loss=0.15518859224532966, metrics={'train_runtime': 137.5948, 'train_samples_per_second': 155.311, 'train_steps_per_second': 4.869, 'total_flos': 1070012411245680.0, 'train_loss': 0.15518859224532966, 'epoch': 5.0})

In [124]:
def convert_list_dict_to_dict_list(
    list_dict: dict[str, list]
) -> list[dict[str, list]]:
    """ミニバッチのデータを事例単位のlistに変換"""
    dict_list = []
    # dictのキーのlistを作成する
    keys = list(list_dict.keys())
    for idx in range(len(list_dict[keys[0]])): # 各事例で処理する
        # dictの各キーからデータを取り出してlistに追加する
        dict_list.append({key: list_dict[key][idx] for key in keys})
    return dict_list

# ミニバッチのデータを事例単位のlistに変換する
list_dict = {
    "input_ids": [[0, 1], [2, 3]],
    "labels": [[1, 2], [3, 4]],
}
dict_list = convert_list_dict_to_dict_list(list_dict)
print(f"入力: {list_dict}")
print(f"出力: {dict_list}")

入力: {'input_ids': [[0, 1], [2, 3]], 'labels': [[1, 2], [3, 4]]}
出力: [{'input_ids': [0, 1], 'labels': [1, 2]}, {'input_ids': [2, 3], 'labels': [3, 4]}]


In [125]:
 list(list_dict.keys())

['input_ids', 'labels']

In [128]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import PreTrainedModel

def run_prediction(
    dataloader: DataLoader, model: PreTrainedModel
) -> list[dict[str, Any]]:
    """予測スコアに基づき固有表現ラベルを予測"""
    predictions = []
    for batch in tqdm(dataloader): # 各ミニバッチを処理する
        inputs = {
            k: v.to(model.device)
            for k, v in batch.items()
            if k != "special_tokens_mask"
        }
        # 予測スコアを取得する
        logits = model(**inputs).logits
        # 最もスコアの高いIDを取得する
        batch["pred_label_ids"] = logits.argmax(-1)
        batch = {k: v.cpu().tolist() for k, v in batch.items()}
        # ミニバッチのデータを事例単位のlistに変換する
        predictions += convert_list_dict_to_dict_list(batch)
    return predictions

# ミニバッチの作成にDataLoaderを用いる
validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator,
)
# 固有表現ラベルを予測する
predictions = run_prediction(validation_dataloader, model)
print(predictions[0]["pred_label_ids"])

100%|██████████| 17/17 [00:01<00:00, 14.97it/s]

[0, 0, 15, 16, 0, 0, 13, 14, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 14, 14, 14, 13, 0, 0, 0, 0, 15, 15, 15, 16, 15, 13, 14, 14, 14, 13, 13, 13, 13, 14, 14, 0, 0, 0, 0, 13, 14, 14, 14, 0, 0, 13, 14, 14, 0, 0, 0, 0, 0, 0, 15, 16, 16, 0, 13, 14, 14, 14, 14, 0, 0, 0, 0, 15, 15, 16, 0, 0, 13, 14]


In [130]:

def extract_entites(predictions, dataset, tokenizer, id2label):
    """
    固有表現の抽出
    """
    results = []
    for prediction, data in zip(predictions, dataset):
    
        # 1文字のリスト化
        characters = list(data['text'])
    
        tokens, pred_labels = [], []
        all_tokens = tokenizer.convert_ids_to_tokens(prediction['input_ids'])
    
        for token, label_id in zip(all_tokens, prediction['pred_label_ids']):
            
            # スペシャルトークンを除く
            if token not in tokenizer.all_special_tokens:
                tokens.append(token)
                pred_labels.append(id2label[label_id])
    
    
            # 文字リストとトークンのListのアライメントをとる
            _, token_to_char_indices = get_alignments(characters, tokens)


            
    
            pred_entities = []

            
            for entity in get_entities(pred_labels):
                print(entity)
                print(token_to_char_indices)
                entity_type, token_start, token_end = entity
                char_start = token_to_char_indices[token_start][0]
                char_end = token_to_char_indices[token_end][-1] + 1
                pred_entity = {
                    'name': "".join(characters[char_start: char_end]), 
                    'span': [char_start, char_end],
                    'type': entity_type,
                }
                print(pred_entity)
    
                pred_entities.append(pred_entity)
            data['pred_entities'] = pred_entities
            results.append(data)
        return results


results = extract_entites(predictions, dataset['validation'], tokenizer, id2label)

            
            


('_', 0, 0)
[[0], [1, 2]]
{'name': '「', 'span': [0, 1], 'type': '_'}
('製品名', 1, 1)
[[0], [1, 2]]
{'name': '復活', 'span': [1, 3], 'type': '製品名'}
('_', 0, 0)
[[0], [1, 2], [3]]
{'name': '「', 'span': [0, 1], 'type': '_'}
('製品名', 1, 2)
[[0], [1, 2], [3]]
{'name': '復活篇', 'span': [1, 4], 'type': '製品名'}
('_', 0, 0)
[[0], [1, 2], [3], [4]]
{'name': '「', 'span': [0, 1], 'type': '_'}
('製品名', 1, 2)
[[0], [1, 2], [3], [4]]
{'name': '復活篇', 'span': [1, 4], 'type': '製品名'}
('_', 0, 0)
[[0], [1, 2], [3], [4], [5]]
{'name': '「', 'span': [0, 1], 'type': '_'}
('製品名', 1, 2)
[[0], [1, 2], [3], [4], [5]]
{'name': '復活篇', 'span': [1, 4], 'type': '製品名'}
('_', 0, 0)
[[0], [1, 2], [3], [4], [5], [6, 7, 8, 9]]
{'name': '「', 'span': [0, 1], 'type': '_'}
('製品名', 1, 2)
[[0], [1, 2], [3], [4], [5], [6, 7, 8, 9]]
{'name': '復活篇', 'span': [1, 4], 'type': '製品名'}
('_', 3, 4)
[[0], [1, 2], [3], [4], [5], [6, 7, 8, 9]]
{'name': '」は', 'span': [4, 6], 'type': '_'}
('法人名', 5, 5)
[[0], [1, 2], [3], [4], [5], [6, 7, 8, 9]]
{'name'

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [131]:
from seqeval.metrics.sequence_labeling import get_entities
from transformers import PreTrainedTokenizer

def extract_entities(
    predictions: list[dict[str, Any]],
    dataset: list[dict[str, Any]],
    tokenizer: PreTrainedTokenizer,
    id2label: dict[int, str],
) -> list[dict[str, Any]]:
    """固有表現を抽出"""
    results = []
    for prediction, data in zip(predictions, dataset):
        # 文字のlistを取得する
        characters = list(data["text"])

        # 特殊トークンを除いたトークンのlistと予測ラベルのlistを取得する
        tokens, pred_labels = [], []
        all_tokens = tokenizer.convert_ids_to_tokens(
            prediction["input_ids"]
        )
        for token, label_id in zip(
            all_tokens, prediction["pred_label_ids"]
        ):
            # 特殊トークン以外をlistに追加する
            if token not in tokenizer.all_special_tokens:
                tokens.append(token)
                pred_labels.append(id2label[label_id])

        # 文字のlistとトークンのlistのアライメントをとる
        _, token_to_char_indices = get_alignments(characters, tokens)

        # 予測ラベルのlistから固有表現タイプと、
        # トークン単位の開始位置と終了位置を取得して、
        # それらを正解データと同じ形式に変換する
        pred_entities = []
        for entity in get_entities(pred_labels):
            entity_type, token_start, token_end = entity
            # 文字単位の開始位置を取得する
            char_start = token_to_char_indices[token_start][0]
            # 文字単位の終了位置を取得する
            char_end = token_to_char_indices[token_end][-1] + 1
            pred_entity = {
                "name": "".join(characters[char_start:char_end]),
                "span": [char_start, char_end],
                "type": entity_type,
            }
            pred_entities.append(pred_entity)
        data["pred_entities"] = pred_entities
        results.append(data)
    return results

# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["validation"], tokenizer, id2label
)
pprint(results[0])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'curid': '1662110',
 'entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
              {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'pred_entities': [{'name': '「', 'span': [0, 1], 'type': '_'},
                   {'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
                   {'name': '」は', 'span': [4, 6], 'type': '_'},
                   {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'text': '「復活篇」はグリーンバニーからの発売となっている。'}


In [92]:
predictions[0]

{'input_ids': [2,
  395,
  13887,
  4436,
  396,
  465,
  14895,
  31227,
  7053,
  12488,
  464,
  12627,
  458,
  12493,
  456,
  12483,
  385,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'special_tokens_mask': [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [138]:
# モデルを読み込む
model_name = "llm-book/bert-base-japanese-v3-ner-wikipedia-dataset"
best_model = AutoModelForTokenClassification.from_pretrained(
    model_name
)
best_model = best_model.to("cuda:0")

In [142]:
# テストセットに対して前処理を行う
test_dataset = dataset["test"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["test"].column_names,
)
# ミニバッチの作成にDataLoaderを用いる
test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator,
)
# 固有表現ラベルを予測する
predictions = run_prediction(test_dataloader, best_model)
# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)
# 正解データと予測データのラベルのlistを作成する
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を出力する
print(classification_report(true_labels, pred_labels))

Map:   0%|          | 0/535 [00:00<?, ? examples/s]

100%|██████████| 17/17 [00:05<00:00,  3.28it/s]


              precision    recall  f1-score   support

           _       0.00      0.00      0.00         0
     その他の組織名       0.86      0.83      0.84       100
       イベント名       0.82      0.92      0.87        93
          人名       0.96      0.96      0.96       287
          地名       0.85      0.86      0.86       204
      政治的組織名       0.75      0.89      0.81       106
         施設名       0.85      0.85      0.85       137
         法人名       0.89      0.88      0.89       248
         製品名       0.76      0.81      0.79       158

   micro avg       0.47      0.88      0.61      1333
   macro avg       0.75      0.78      0.76      1333
weighted avg       0.86      0.88      0.87      1333



In [143]:
results

[{'curid': '49396',
  'text': '統治機構の近代化により王朝を立て直すことに失敗、加えて義和団の乱後をめぐる清朝の醜態も加わり、1911年の辛亥革命への機運が高まる。',
  'entities': [{'name': '義和団の乱', 'span': [27, 32], 'type': 'イベント名'},
   {'name': '清朝', 'span': [37, 39], 'type': '政治的組織名'},
   {'name': '辛亥革命', 'span': [53, 57], 'type': 'イベント名'}],
  'pred_entities': [{'name': '統治機構の近代化により王朝を立て直すことに失敗、加えて',
    'span': [0, 27],
    'type': '_'},
   {'name': '義和団の乱', 'span': [27, 32], 'type': 'イベント名'},
   {'name': '後をめぐる', 'span': [32, 37], 'type': '_'},
   {'name': '清朝', 'span': [37, 39], 'type': '政治的組織名'},
   {'name': 'の醜態も加わり、1911年の', 'span': [39, 53], 'type': '_'},
   {'name': '辛亥革命', 'span': [53, 57], 'type': 'イベント名'}]},
 {'curid': '3352215',
  'text': '奏者は普通、爪を用いて演奏する。',
  'entities': [],
  'pred_entities': []},
 {'curid': '1755276',
  'text': "株式会社ナムコに入社し、ファミスタ'88等の作曲を手掛ける。",
  'entities': [{'name': '株式会社ナムコ', 'span': [0, 7], 'type': '法人名'},
   {'name': "ファミスタ'88", 'span': [12, 20], 'type': '製品名'}],
  'pred_entities': [{'name': '株式会社ナムコ', 'span': [

In [150]:
a = {'entities': [{'name': '義和団の乱', 'span': [27, 32], 'type': 'イベント名'},
   {'name': '清朝', 'span': [37, 39], 'type': '政治的組織名'},
   {'name': '辛亥革命', 'span': [53, 57], 'type': 'イベント名'}]}

b = {'pred_entities': [{'name': '統治機構の近代化により王朝を立て直すことに失敗、加えて',
    'span': [0, 27],
    'type': '_'},
   {'name': '義和団の乱', 'span': [27, 32], 'type': 'イベント名'},
   {'name': '後をめぐる', 'span': [32, 37], 'type': '_'},
   {'name': '清朝', 'span': [37, 39], 'type': '政治的組織名'},
   {'name': 'の醜態も加わり、1911年の', 'span': [39, 53], 'type': '_'},
   {'name': '辛亥革命', 'span': [53, 57], 'type': 'イベント名'}]}

a['entities'] == b['pred_entities']

False

In [152]:
def find_error_results(
    results: list[dict[str, Any]],
) -> list[dict[str, Any]]:
    """エラー事例を発見"""
    error_results = []
    for idx, result in enumerate(results): # 各事例を処理する
        result["idx"] = idx
        # 正解データと予測データが異なるならばlistに加える
        if result["entities"] != result["pred_entities"]:
            error_results.append(result)
    return error_results

def output_text_with_label(result: dict[str, Any], entity_column: str) -> str:
    """固有表現ラベル付きテキストを出力"""
    text_with_label = ""
    entity_count = 0
    for i, char in enumerate(result["text"]): # 各文字を処理する
        # 出力に加えていない固有表現の有無を判定する
        if entity_count < len(result[entity_column]):
            entity = result[entity_column][entity_count]
            # 固有表現の先頭の処理を行う
            if i == entity["span"][0]:
                entity_type = entity["type"]
                text_with_label += f" [({entity_type}) "
            text_with_label += char
            # 固有表現の末尾の処理を行う
            if i == entity["span"][1] - 1:
                text_with_label += "] "
                entity_count += 1
        else:
            text_with_label += char
    return text_with_label

# エラー事例を発見する
error_results = find_error_results(results)
# 3件のエラー事例を出力する
for result in error_results[:3]:
    idx = result["idx"]
    true_text = output_text_with_label(result, "entities")
    pred_text = output_text_with_label(result, "pred_entities")
    print(f"事例{idx}の正解: {true_text}")
    print(f"事例{idx}の予測: {pred_text}")
    print()

事例0の正解: 統治機構の近代化により王朝を立て直すことに失敗、加えて [(イベント名) 義和団の乱] 後をめぐる [(政治的組織名) 清朝] の醜態も加わり、1911年の [(イベント名) 辛亥革命] への機運が高まる。
事例0の予測:  [(_) 統治機構の近代化により王朝を立て直すことに失敗、加えて]  [(イベント名) 義和団の乱]  [(_) 後をめぐる]  [(政治的組織名) 清朝]  [(_) の醜態も加わり、1911年の]  [(イベント名) 辛亥革命] への機運が高まる。

事例2の正解:  [(法人名) 株式会社ナムコ] に入社し、 [(製品名) ファミスタ'88] 等の作曲を手掛ける。
事例2の予測:  [(法人名) 株式会社ナムコ]  [(_) に入社し、]  [(製品名) ファミスタ'88] 等の作曲を手掛ける。

事例3の正解: 1947年、移行活動がまだ進行中であったため当時の [(施設名) ペイン陸軍飛行場] の軍事管理は [(政治的組織名) アメリカ陸軍航空軍] の後身である [(政治的組織名) アメリカ空軍] に移管され、空港は [(施設名) ペイン・フィールド] と改名された。
事例3の予測:  [(_) 1947年、移行活動がまだ進行中であったため当時の]  [(施設名) ペイン陸軍飛行場]  [(_) の軍事管理は]  [(政治的組織名) アメリカ陸軍航空軍]  [(_) の後身である]  [(政治的組織名) アメリカ空軍]  [(_) に移管され、空港は]  [(施設名) ペイン・フィールド] と改名された。



In [ ]:
re